# Part 3 of the Investigation into "their kingdom" of 4Q246 II.2

<img src="images/their_kingdom.png"></img>

In [1]:
from datetime import datetime
print(f'Notebook last updated on {datetime.now().__str__()}')

Notebook last updated on 2017-05-24 16:03:23.982911


# Introduction

In the two previous notebooks, `their_kingdom_1.ipynb` and `their_kingdom_2.ipynb`, we explored two issues related to the identification of the antecedent for the 3MP suffix on מלכותהן ("their kingdom"), a kingdom that the text develops as temporary and destructive. Does the 3MP refer to those who name the "son of God," thereby tying him in with the bad kingdom? In `their_kingdom_1.ipynb` we found that the conditions are present in 4Q246 for a disconnect between the 3MP of "they will name him" and "their kingdom," due to the parallel construction with a passive verb and the use of an impersonal subject. This raised the possibility of a different antecedent. One attractive option in col. 1 is an apparent reference to a "king(s) of Assyria and Egypt," but the question remained whether that expression could indeed be plural. The second notebook, `their_kingdom_2.ipynb`, looked for plural verbs with singular subjects in construct with coordinated nomen recta. It found parallels in Gen 14:10 and Isaiah 63:15, but the construction is clearly rare. 

## Research Question
**But what are the conditions that give rise to a suffix-antecedent connection that spans several clauses?** Is such a scenario rare?

That is the question which motivates the present notebook. The answer has the potential to not only influence the investigation into the 3MP of "their kingdom," but also the interpretation of the 3MS on "his/its/their kingdom" (מלכותה, col.2 ln. 5), which remains controversial as well, and for which pro-messianic readings propose another suffix-antecedent connection that spans multiple clauses.

## Goal
Get instances from the HB wherein: 
1. a clause atom's mother is several clauses away
2. a third person suffix is attached to a subject
3. the clause atom's mother contains a noun which agrees with the clause's suffixed subject.

Then export the instances to a spreadsheet to be manually labeled whether the suffix antecedent and the substantive antecedent is the same. 

Finally, analyze the circumstances which surround the connections. **What kinds of features are present in such a distance construction? What kinds of elements help disambiguate or clarify the distant connection?**

## Functions

The functions from the `participant_functions` directory, specifically `pgn.py` and `subjects.py`, will be important for this notebook.

Also the following is important:
* `dist` - [distance to mother](https://etcbc.github.io/text-fabric-data/features/hebrew/etcbc4c/dist), to determine that a mother clause is N away. The feature documentation urges caution for this feature, since it is inconsistently applied, and advises that the distance be calculated directly by taking the difference between the mother and daughter. We will do this in the code.

* the referent "king(s) of Assyria and Egypt" occurs about 9 clauses away from the suffix. The 3MS referent for "his/its kingdom" occurs 3 clause atoms away. We will start with clause atoms with a distance from their mothers of 3. If those results are too numerous, we will gradually extend out moving up to 8.

In [2]:
# begin code

# import tools
import collections, csv

# import text fabric
from tf.fabric import Fabric

# instantiate TF processor
TF = Fabric(modules='hebrew/etcbc4c', silent=True)

# load TF features
api = TF.load('''
              book chapter verse
              prs prs_nu prs_ps prs_gn
              ps gn nu
              mother function pdp 
              typ rela tab domain
              ''')

# prevent having to prefix all functions with .api
# also necessary for the pgn.py and subject.py functions
api.makeAvailableIn(globals())

  0.00s loading features ...
   |     0.01s B book                 from /Users/Cody/github/text-fabric-data/hebrew/etcbc4c
   |     0.01s B chapter              from /Users/Cody/github/text-fabric-data/hebrew/etcbc4c
   |     0.01s B verse                from /Users/Cody/github/text-fabric-data/hebrew/etcbc4c
   |     0.12s B prs                  from /Users/Cody/github/text-fabric-data/hebrew/etcbc4c
   |     0.12s B prs_nu               from /Users/Cody/github/text-fabric-data/hebrew/etcbc4c
   |     0.12s B prs_ps               from /Users/Cody/github/text-fabric-data/hebrew/etcbc4c
   |     0.12s B prs_gn               from /Users/Cody/github/text-fabric-data/hebrew/etcbc4c
   |     0.12s B ps                   from /Users/Cody/github/text-fabric-data/hebrew/etcbc4c
   |     0.10s B gn                   from /Users/Cody/github/text-fabric-data/hebrew/etcbc4c
   |     0.12s B nu                   from /Users/Cody/github/text-fabric-data/hebrew/etcbc4c
   |     0.18s B mother        

Import special, custom functions for subject and person, gender, number matching.

In [3]:
import participant_functions.pgn as pgn
import participant_functions.subjects as subj

## Code

Now we compose the query and run it.

In [20]:
def match_distant_referent(ca, min_dist=3):
    
    '''
    Return dict data on a clause atom if it is a match for:
        * its mother is several clause atoms away
        * a third person suffix is attached to its subject
        * its mother contains a noun which agrees 
            with the clause atom's suffixed subject.
    
    Require a clause atom node number.
    Optional parameter for min_dist for the minimum distance of
        a clause atom from its mother.
    '''
    
    
    # /1/ Check parameters for target clause atom //
    
    # get mother
    mother = E.mother.f(ca)[0] if E.mother.f(ca) else None
    
    # no match if no mother
    if not mother:
        return None
    
    # check distance to mother against min_distance
    distance = ca - mother
    # no match if not min
    if distance < min_dist:
        return None
    
    # check/get subject(s) word (to check for suffixes)
    subject = tuple(word for phrase in L.d(ca, otype='phrase')
                       for word in L.d(phrase, otype='word')
                    
                       # word must be in subj phrase
                       if F.function.v(phrase) == 'Subj'
                    
                       # word must be the subject noun
                       and subj.validate_subject(word)
                    )
    
    # don't match plural or ø subjects
    if len(subject) > 1 or not subject:
        return None
    
    # isolate subject from tuple
    subject = subject[0]
    
    # don't match ø pronominal suffix or non-third person pr.sffx.
    if F.prs.v(subject) in {'n/a','absent'} or F.prs_ps.v(subject) != 'p3':
        return None
    
    # get PGN data for pronominal suffix
    prs_pgn = pgn.get_pgn(subject, pronom=True)
    
    
    # /2/ Check parameters for mother clause atom //
    
    # get the subject from the mother clause
    mother_subj = tuple(word for phrase in L.d(mother, otype='phrase')
                           for word in L.d(phrase, otype='word')
                           if F.function.v(phrase) == 'Subj' # word in subj phrase
                           and subj.validate_subject(word) # word is subject noun
                        )
    
    # don't match plural or ø mother subjects
    if len(mother_subj) > 1 or not mother_subj:
        return None
    
    # isolate mother subject from the tuple
    mother_subj = mother_subj[0]
    
    # get the PGN for the mother's subject
    mother_pgn = pgn.get_pgn(mother_subj)
    
    
    # /3/ compare the target clause atom parameters with its mother's //
    
    # match if pgn of suffix and mother subject agree
    if pgn.match_pgn(prs_pgn, mother_pgn):
        
        # ASSEMBLE MATCH DATA
        
        # format reference
        book, chapter, verse1, = T.sectionFromNode(ca)
        verse2 = T.sectionFromNode(mother)[2]
        reference = f'{book} {chapter}:{verse1}' if verse1 == verse2\
                        else f'{book} {chapter}:{verse1}-{verse2}'

        # assemble match text
        ca_words = L.d(ca, otype='word') # get target clause atom words
        mother_words = L.d(mother, otype='word') # get mother ca words
        ca_text, mo_text = T.text(ca_words), T.text(mother_words) # convert words to text
        match_text = f'{mo_text}\n...\n{ca_text}' # assemble into string

        # get suffix/subj text
        sfx_text = T.text([subject])
        sbj_text = T.text([mother_subj])


        # get all the text from mother to daughter to aid manual sorting

        # get get all intervening clause atoms
        all_cas = range(mother-5, ca+6) # -2 and +3 to catch a bit before and after

        # assemble clause atom text to this list
        # include indentation data for analyzing the clause hierarchy
        # TODO: clean up this code with more separation and notation
        cas_text = []
        cas_types = []
        cas_domains = []
        for cla in all_cas:
            marker = '*' if cla in {mother, ca} else ''
            indent = '–' * F.tab.v(cla)
            ca_words = L.d(cla, otype='word') # words for plain text
            ca_text = T.text(ca_words) # get plain text
            cas_text.append(f'{ca_text}{marker}{indent}')
            cas_types.append(F.typ.v(cla))
            cas_domains.append(F.domain.v(L.u(cla, otype='clause')[0]))
        context_text = '\n'.join(cas_text)    
        context_types = '\n'.join(cas_types)
        context_domains = '\n'.join(cas_domains)

        # return match data
        return {'reference':reference,
                'match_text':match_text,
                'distance':distance,
                'same?':'', # field for manual notations
                'notes':'', # field for manual notations
                'suffix':sfx_text,
                'subject':sbj_text,
                'sfx_pgn':prs_pgn,
                'sbj_pgn':mother_pgn,
                'context':context_text,
                'con_typs':context_types,
                'con_doms':context_domains,
                'ca_typ':F.typ.v(ca),
                'mo_typ':F.typ.v(mother),
                'ca':ca,
                'mother':mother
               }
        
    # not a match
    else:
        return None

### Run the Query in the HB



In [21]:
results = []

# find matches from all clause atoms in HB
for clause_atom in F.otype.s('clause_atom'):
    
    # run the match function
    is_match = match_distant_referent(clause_atom)
    
    # append matches to results
    if is_match:
        results.append(is_match)
        
    else:
        continue
        
print(f'Done with {len(results)} results!')

Done with 113 results!


Now we export the results to a csv file for manual processing...

In [22]:
fields = ('reference','match_text',
          'distance','same?',
          'notes','suffix',
          'subject','sfx_pgn',
          'sbj_pgn','context','con_typs',
          'con_doms','ca_typ','mo_typ',
          'ca','mother')

export_file = 'results/distant_suffixes_HB.csv'

# export commented out to protect written files
with open(export_file, 'w') as outfile:
    
    writer = csv.DictWriter(outfile, fieldnames=fields)
    
    writer.writeheader()
    writer.writerows(results)

[**View the results.**](results/distant_suffixes_HB.csv)